# 构建词图
将文本中的词语作为节点，词语之间的邻接关系作为边，构成的有向带权图，称为词图。词图本质上为一个二元组：$G = (V，E )$，其中，$V$ 为词语构成的顶点集，$E$为词语相邻形成的边集。

## 分词和词性标记
在词图构建过程中，给定一段文本，首先对其进行**分词**和**词性标记**，形成原始分词序列，记为 $L1={ w_1/t_1，w_2/t_2，⋯，w_n/t_n}$，$t_i$ 为词语 $w_i$ 的词性标记；

## 分词序列过滤
其次，对 $L_1$进行过滤，仅保留词性为**名词**（包括普通名词、人名、地名、机构名、专有名词）、**动词**和**形容词**的三类词语，将过滤后的序列记为 $L_2$；

## 相邻加边
对于任意的一对词语$(w_a，w_b)$，如 $w_a$和 $w_b$前后相邻出现在$L_2$中，则在词图$G$中设置一条由$w_a$指向$w_b$的边。

**潜在关键短语**：为方便关键短语的抽取，如果$(wa，wb)$在原始序列$L1$中同样相邻出现，且$w_b$对应的词性为名词，则$(w_a，w_b)$ 为潜在的关键短语，作为关键短语识别的处理对象进行后续计算分析。

## 词图片段
- **节点内左侧**文字表示词语，**右侧数字**表示该词语在文本中出现的频度；
- **实线**相连的两个词语表示在文本中前后相邻，是候选短语的选择来源；
- **虚线**则表示两个词语之间还存在其他词语，如连词、助词、介词等，相连词语会参与到候选短语的重要性计算之中。

# 词语节点排序
词语节点排序是通过一定算法计算词语在词图网络中的重要性。将词语根据重要性大小降序排序，从而获得一组具有高影响力、能够反映文本核心概念的词语结果。单个词语本身也可以构成关键短语，一个词语在词图中起到的作用越大，其成为关键短语的概率就越大。

$TextRank$是词图当中度量词语重要性的主要方法，该方法以迭代方式按照以下公式计算节点 $v_i$的重要性：

$S\left(v_{i}\right)=(1-\lambda)+\lambda \sum_{j \in N\left(v_{i}\right)} \frac{1}{N\left(v_{j}\right)} S\left(v_{j}\right)$ 

其中，$N (v_i)$ 表示节点 $v_i$的相邻节点集合；$S (v_j)$表示节点 $v_j$的重要性得分；$λ$ 为阻尼系数，通常取值为 0.85，在公式中表示任意时刻都有 $1 - λ$ 的概率由当前节点随机跳转到词图中的其他节点之上。当计算收敛之后，$S (v_i) $值可以表示节点$ v_i$在词图中的重要性。

在实践中，$TextRank$算法还可以将词语频度、长度、位置等信息纳入到计算过程之中，进一步提升词语节点的排序效果。本文基于前期在$TextRank$方面的相关研究，采用位置加权方法进行节点重要性计算和排序。

令向量 $B_i$为第 $i$ 轮迭代时每一个节点的重要性得分，概率转移矩阵 $M$ 中的每一个元素 $e_i$，$j$表示由词语节点 $i$ 跳转到词语节点 $j$ 的概率，$M$ 的具体取值通过位置加权策略确定，则有公式:

$\boldsymbol{B}_{i}=(1-\lambda) \frac{e}{n}+\lambda \cdot \boldsymbol{M} \cdot \boldsymbol{B}_{i-1}$

基于以上公式进行迭代计算，当计算结果收敛之后，向量$ B $中的第 $ i $ 个元素即为词语节点 $ i $ 的重要性得分，为词语排序提供判断依据。

# 双词短语排序

## 基本假设
给定两个独立的词语 a、b，令 a → b 表示 a、b 在文本中前后紧邻出现的一个词语对，即在词图中存在由 a指向 b的边，则 a → b构成短语的可能性与如下因素有关。
- ①假设 1：词语 a、b 前后直接相邻出现的次数越多，则 a → b 越有可能成为一个有代表意义的关键短语。
- ②假设2：长度较短的词语与长度较长的词语相比，具有更高的与其他词语组合成为关键短语的能力。
- ③假设 3：相比其他词语节点，词语 a、b 结合越紧密，则a → b构成关键短语的可能性越大。
- ④假设 4：词语 a 的左侧相邻的词语种类越多，则词语 a 越不易于和左侧的其他词语构成关键短语，因此，从 a 的左侧断开，a 与右侧的词语 b 构成短语的可能性越高。
- ⑤假设 5：词语 b 的右侧相邻的词语种类越多，则词语 b 越不易于和其右侧的其他词语构成关键短语，因此，从 b 的右侧断开，b 与左侧的词语 a 构成短语的可能性越高。

图 3给出了以词语 a、b为中心的词图片断，边的方向表明两个词语的前后出现顺序，边的粗细代表词对出现的频度大小，虚线表示由其他词语节点指向节点 a 或 b，实线则表示由节点 a 或 b 指向其他词语节点。

为便于表述，令 phrase (a → b) 表示 a、b合在一起构成关键短语的概率大小。

## 内部凝聚度计算
为反映两个词语构成短语的内部紧密程度，本文提出凝聚度指标（Cohesion Indicator），基于前三个假设计算凝聚度大小。

首先，给定由单个词语作为节点构成的词图，假设 1可通过词语节点 $a$、$b$相邻出现的频度度量，假设2 则表明 $a → b$ 构成短语的可能性与两个词语的长度成反比。因此，可得出如下关系：

$phrase (a \rightarrow b) \propto \operatorname{Count}(a \rightarrow b) \frac{1}{|a||b|}$

其中，$phrase (a → b)$ 表示 a、b 合在一起构成关键短语的概率；$Count (a → b)$ 表示词语 a、b 共现的频度；$|a|$和$|b|$分别表示构成词语$a$、$b$的字符个数。

其次，假设 3用于反映一对词语 $a$、$b$与其关联节点相比，构成关键短语的相对能力大小。在由 a或 b构成的所有潜在短语中，a → b 的相对比重越大，则$a → b $ 构成短语的稳固性越强，这一特点可表示如公式（4）：

$phrase (a \rightarrow b) \propto p\left(a \rightarrow b \mid S_{\text {out }}(a)\right) p\left(a \rightarrow b \mid S_{\text {in }}(b)\right)$

其中，$S_{out}( a )$ 表示经节点 a 出来指向其他词语节点的相关节点集合，图 3中为节点 a右侧的三条出边所指向的目标节点；$S_{in}( b )$ 表示从其他节点指向节点 b 的所有节点集合，图 3 中为节点 b 左侧的 4 条入边对应的源节点；$p (a → b|S_{out} (a ) )$ 表示词图节点$a $在与其他节点向右聚合形成 $a → x$ 的词对之时，属于 $a → b$ 这一情况的概率，该值越大，说明 $a$ 与右侧的 $b$ 结 合 为 短 语 的 程 度 越 紧 密 ；同 理 ，$p ( a → b|S_{in} ( b ) ) $ 用 于 度 量 节 点 $b$ 的 左 聚 合 能 力 ，$a → b$ 在 $b$ 的所有左聚合情况中，概率越大，则构成短语的可能性越大。

基于极大似然估计，得到公式（5）和公式（6）。

$p\left(a \rightarrow b \mid S_{\text {out }}(a)\right)=\frac{\operatorname{Count}(a \rightarrow b)}{\sum_{n \in S_{\text {out }}(a)} \operatorname{Count}(a \rightarrow n)}$

$p\left(a \rightarrow b \mid S_{\text {in }}(b)\right)=\frac{\operatorname{Count}(a \rightarrow b)}{\sum_{n \in S_{\text {in }(b)}} \operatorname{Count}(n \rightarrow b)}$

综合公式（3）-公式（6），可得出如下关系。

$phrase (a \rightarrow b) \propto \frac{\operatorname{Count}(a \rightarrow b)}{|a||b|} \cdot \frac{\operatorname{Count}(a \rightarrow b)}{\sum_{n \in S_{out}(a)} \operatorname{Count}(a \rightarrow n)} \cdot \frac{\operatorname{Count}(a \rightarrow b)}{\sum_{n \in S_{in}(b)} \operatorname{Count}(n \rightarrow b)}$

由于关键短语的抽取结果取决于每个短语得分的相对大小，对公式（7）取对数，不改变排序结果的顺序，为方便运算，本文将 a → b 的内部凝聚度cohesion ( a → b ) 定义为公式（7）的对数运算结果，将公式（7）右侧取对数并化简，可得到如公式（8）所示的内部凝聚度计算公式。
$\operatorname{cohesion}(a \rightarrow b)=3 \log _{2} \operatorname{Count}(a \rightarrow b)-\log _{2}|a|-\log _{2}|b|-\log _{2} \sum_{n \in S_{o u t}(a)} \operatorname{Count}(a \rightarrow n)-\log _{2} \sum_{n \in S_{m}(b)} \operatorname{Count}(n \rightarrow b)$

## 边界自由度计算

自由度指标（Freedom Indicator）用于度量一个词语与相邻词语自由搭配的能力。基于假设 4 和假设 5，将自由度进一步分为左自由度和右自由度，其中，左自由度指一个词语与左侧的其他词语自由组合的能力，右自由度则指与右侧其他词语自由组合的能力。给定两个邻接的词语 a → b，a 的左自由度和 b 的右自由度越高，则 a → b 与两侧相邻词语断开，构成关键短语的可能性越大。

为度量自由度的大小，本文基于信息熵原理，通过公式（9）和公式（10）分别计算词语 $a$ 的左自由度 $freedom_L{( a )}$ 和词语 $b$ 的右自由度 $freedom_R{( b )}$。

其中，对于$∀m ∈ S_{in} (a )$和$∀n ∈ S_{out} (b)$，$p (m → a )$和$ p (b → n) $可根据公式（11）和公式（12）分别计算得到。

$p(m \rightarrow a)=\frac{\operatorname{Count}(m \rightarrow a)}{\sum_{w \in S_{w}(a)} \operatorname{Count}(w \rightarrow a)}$

$p(b \rightarrow n)=\frac{\operatorname{Count}(b \rightarrow n)}{\sum_{w \in S_{o u t}(b)} \operatorname{Count}(b \rightarrow w)}$

本质上，公式（9）- 公式（12）的计算结果为边界词语与其他邻接词语随机出现的熵值，熵作为度量事件不确定性的信息单位，其取值反映了词语的自由搭配能力。 进一步，a → b的边界自由度freedom ( a → b ) 可通过 a 的左自由度和 b 的右自由度合并计算得到，如公式（13）所示:

$freedom (a \rightarrow b)= freedom _{L}(a)+ freedom _{R}(b)$

以图3为例，指向词语节点 $a$ 的入边越多且频度越均匀，则左侧词语与 $a$ 合并形成关键短语的概率越低；词语节点 $b$ 的出边越多且频度越均匀，则 $b$ 与其右侧相邻词语构成短语的概率越低。此时，a → b 拥有更高的概率构成关键短语。

## 短语权威度计算与排序
输出两个相邻词语a → b 对应的短语权威度由其内部凝聚度和两侧的边界自由度共同决定，基于公式（14）运算得到。
$\begin{aligned} \text { phrase }(a \rightarrow b) = \alpha \cdot \text { cohesion }(a \rightarrow b)+ \beta \cdot \text { freedom }(a \rightarrow b) \end{aligned}$

其中，α、β为可调节的参数，0 < α < 1，0 < β < 1，且 α + β = 1。α 越大，表明短语最终的权威度更大程度上依赖于词语 a、b 的内部凝聚度；否则，越依赖于词语两侧的边界自由度。本文后续实验中，α 和 β取值均为0.5。

对所有符合条件的邻接词语对计算权威度，按照权威度由高到低排序，输出 $TopN$ 个结果作为关键短语参与后续的序列融合处理。

# 序列融合
基于短语权威度排序和节点权威度排序，共得到两个独立的序列，即基于加权 $TextRank$ 得到的词语权威度输出序列和基于短语权威度运算得到的短语序列，此时，还需要进一步将两个独立序列融合排序，在单一序列上获取最终的抽取结果。

令 $S_p= { p_1，p_2，⋯，p_m}$表示抽取得到的 $m$ 个按照短语权威度降序排列的候选短语 ；$S_w={ w_1，w_2，⋯，w_n}$表示按照词语权威度大小降序排列的 n个词语；$S ∈ { S_p，S_w}$；$r ( x|S ) $表示 $x$在结果集 $S$中的排序位置，即结果集 S 中第 1 个元素的$ r$ 值为 1，第2 个为 2，以此类推。作为$ x$ 的排序重要性，$r ( x|S ) $取值越小，则在排序结果中越重要。如果元素 $x ∉ S$，则令$r ( x|S ) = ∞$。按照以下步骤进行序列融合。

## 无效候选短语的过滤
仅当候选短语 $p_i$中的每一个词语都属于集合$S_w$，$p_i$为可接受短语，否则，将 $p_i$从 $S_p$中过滤掉，以保证构成关键短语的每一个词语本身在词图网络中也具有较高的权威度。